# INTERNPE INTERNSHIP, OCTOBER 2023

# ARTIFICIAL INTELLIGENCE AND MACHINE LEARNING INTERNSHIP

## PROBLEM STATEMENT

### Make a IPL Winning Team Prediction

## SOLUTION

## AUTHOR : ARITRA BAG

#### IMPORTING LIBRARIES

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

#### IMPORTING THE DATASETS

In [91]:
#Reading dataset from a .CSV file directly
match_results=pd.read_csv('/home/aritrabag/Documents/internship_projects/iplDatasets/matches.csv')
match_results.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [93]:
#Reading dataset from a .CSV file directly
match_summary=pd.read_csv('/home/aritrabag/Documents/internship_projects/iplDatasets/deliveries.csv')
match_summary.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


#### DATA PREPROCESSING

In [94]:
#creating a separate dataframe
processed_summary = match_summary
processed_results = match_results

In [95]:
#setting the target score 
total_innings_score = processed_summary.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
first_innings  = total_innings_score[total_innings_score['inning']==1]
first_innings.head()                                               

,match_id,inning,total_runs
0,1,1,207
2,2,1,184
4,3,1,183
6,4,1,163
8,5,1,157


In [96]:
#appending the first innings score to the match results
processed_results = processed_results.merge(first_innings[['match_id','total_runs']],left_on='id',right_on='match_id')
#renaming total_runs to targets
processed_results.rename(columns = {'total_runs' : 'target'},inplace = True)
processed_results.reset_index(inplace=True)
processed_results.head()

,index,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,...,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,target
0,0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,...,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
1,1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,...,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,184
2,2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,...,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,3,183
3,3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,...,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,4,163
4,4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,...,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,157


In [97]:
#mergigng the city and venue columns
processed_results['venue']=processed_results['venue'].astype(str)+ ' , ' +processed_results['city']
processed_results.drop(['city'],axis=1,inplace = True)
processed_results.head()

,index,id,Season,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,target
0,0,1,IPL-2017,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal , Hy...",AY Dandekar,NJ Llong,NaN,1,207
1,1,2,IPL-2017,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,"Maharashtra Cricket Association Stadium , Pune",A Nand Kishore,S Ravi,NaN,2,184
2,2,3,IPL-2017,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,"Saurashtra Cricket Association Stadium , Rajkot",Nitin Menon,CK Nandan,NaN,3,183
3,3,4,IPL-2017,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,"Holkar Cricket Stadium , Indore",AK Chaudhary,C Shamshuddin,NaN,4,163
4,4,5,IPL-2017,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,"M Chinnaswamy Stadium , Bangalore",NaN,NaN,NaN,5,157


In [99]:
#removing the first innings results from the match_summary
innings_2_summary = processed_summary[processed_summary['inning']==2]
innings_2_summary.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
125,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,Mandeep Singh,A Nehra,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
127,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
128,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
129,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN


In [100]:
#merging the match_results and match_summary datasets
stats = processed_results[['match_id','venue', 'target','winner','result','dl_applied']]
stats.head()

,match_id,venue,target,winner,result,dl_applied
0,1,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0
1,2,"Maharashtra Cricket Association Stadium , Pune",184,Rising Pune Supergiant,normal,0
2,3,"Saurashtra Cricket Association Stadium , Rajkot",183,Kolkata Knight Riders,normal,0
3,4,"Holkar Cricket Stadium , Indore",163,Kings XI Punjab,normal,0
4,5,"M Chinnaswamy Stadium , Bangalore",157,Royal Challengers Bangalore,normal,0


In [101]:
innings_2_summary = innings_2_summary.merge(stats,on='match_id')
innings_2_summary.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,venue,target,winner,result,dl_applied
0,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,Mandeep Singh,A Nehra,0,...,0,1,NaN,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,NaN,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0
2,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,CH Gayle,A Nehra,0,...,0,0,NaN,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0
3,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,CH Gayle,A Nehra,0,...,0,2,NaN,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0
4,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,CH Gayle,A Nehra,0,...,0,4,NaN,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,normal,0


In [102]:
#selecting only features that matter
innings_2_summary = innings_2_summary[['match_id', 'batting_team', 'bowling_team', 'venue', 'target', 'over', 'ball', 'total_runs', 'player_dismissed', 'dl_applied', 'winner', 'result']]
innings_2_summary.dropna(subset=['match_id', 'batting_team', 'bowling_team', 'venue', 'target', 'over', 'ball', 'total_runs', 'dl_applied', 'winner', 'result'],axis = 0, inplace = True)
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary.head()

,match_id,batting_team,bowling_team,venue,target,over,ball,total_runs,player_dismissed,dl_applied,winner,result
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,1,1,NaN,0,Sunrisers Hyderabad,normal
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,2,0,NaN,0,Sunrisers Hyderabad,normal
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,3,0,NaN,0,Sunrisers Hyderabad,normal
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,4,2,NaN,0,Sunrisers Hyderabad,normal
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,5,4,NaN,0,Sunrisers Hyderabad,normal


In [103]:
#rebranding Delhi Daredevils to Delhi Capitals and Deccan Chargers to Sunrisers Hyderabad after considering them as replacements of their respective formers
innings_2_summary['batting_team'] = innings_2_summary['batting_team'].str.replace('Delhi Daredevils','Delhi Capitals')
innings_2_summary['bowling_team'] = innings_2_summary['bowling_team'].str.replace('Delhi Daredevils','Delhi Capitals')

innings_2_summary['batting_team'] = innings_2_summary['batting_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
innings_2_summary['bowling_team'] = innings_2_summary['bowling_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [104]:
#dropping the franchises that dropped out
print(innings_2_summary['batting_team'].unique())

['Royal Challengers Bangalore' 'Rising Pune Supergiant'
 'Kolkata Knight Riders' 'Kings XI Punjab' 'Delhi Capitals'
 'Sunrisers Hyderabad' 'Mumbai Indians' 'Gujarat Lions' 'Rajasthan Royals'
 'Chennai Super Kings' 'Pune Warriors' 'Kochi Tuskers Kerala'
 'Rising Pune Supergiants']


In [105]:
acceptable_teams = ['Royal Challengers Bangalore', 'Kolkata Knight Riders', 'Kings XI Punjab','Sunrisers Hyderabad', 'Mumbai Indians', 'Rajasthan Royals', 'Chennai Super Kings', 'Delhi Capitals']

In [106]:
innings_2_summary = innings_2_summary[innings_2_summary['batting_team'].isin(acceptable_teams)]
innings_2_summary = innings_2_summary[innings_2_summary['bowling_team'].isin(acceptable_teams)]
print(innings_2_summary['batting_team'].unique())
print(innings_2_summary['bowling_team'].unique())

['Royal Challengers Bangalore' 'Delhi Capitals' 'Mumbai Indians'
 'Kings XI Punjab' 'Kolkata Knight Riders' 'Sunrisers Hyderabad'
 'Rajasthan Royals' 'Chennai Super Kings']
['Sunrisers Hyderabad' 'Royal Challengers Bangalore'
 'Kolkata Knight Riders' 'Kings XI Punjab' 'Delhi Capitals'
 'Mumbai Indians' 'Chennai Super Kings' 'Rajasthan Royals']


In [107]:
#selecting the matches only where no DL method is applied
innings_2_summary = innings_2_summary[innings_2_summary['dl_applied']==0]

#changing the winner to To be Decided by Super-Over in case the result is a tie
innings_2_summary['winner']=innings_2_summary['winner'].astype(str)
innings_2_summary['winner'][innings_2_summary['result']=='tie'] = 'To be Decided by Super-Over'

/tmp/ipykernel_4447/853559085.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  innings_2_summary['winner'][innings_2_summary['result']=='tie'] = 'To be Decided by Super-Over'


In [108]:
#getting the current score, runs left, balls left, required run rate and current run rate
innings_2_summary['total_runs'] = pd.to_numeric(innings_2_summary['total_runs'], errors='coerce')
innings_2_summary['total_runs'] = innings_2_summary['total_runs'].fillna(0)
innings_2_summary['total_runs'] = innings_2_summary['total_runs'].astype(int)
innings_2_summary['over'] = innings_2_summary['over'].astype(int)
innings_2_summary['ball'] = innings_2_summary['ball'].astype(int)

innings_2_summary['current_score'] = innings_2_summary.groupby(by=['match_id'])['total_runs'].transform(lambda x: x.cumsum())
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary['runs_left'] = innings_2_summary['target'] - innings_2_summary['current_score']
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary['balls_left'] = 126 - (innings_2_summary['over']*6 + innings_2_summary['ball'])
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary['rrr'] = (innings_2_summary['runs_left']*6)/innings_2_summary['balls_left']
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary['crr'] = (innings_2_summary['current_score']*6)/(120 - innings_2_summary['balls_left'])
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary.head()

,match_id,batting_team,bowling_team,venue,target,over,ball,total_runs,player_dismissed,dl_applied,winner,result,current_score,runs_left,balls_left,rrr,crr
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,1,1,NaN,0,Sunrisers Hyderabad,normal,1,206,119,10.386555,6.0
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,2,0,NaN,0,Sunrisers Hyderabad,normal,1,206,118,10.474576,3.0
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,3,0,NaN,0,Sunrisers Hyderabad,normal,1,206,117,10.564103,2.0
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,4,2,NaN,0,Sunrisers Hyderabad,normal,3,204,116,10.551724,4.5
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,5,4,NaN,0,Sunrisers Hyderabad,normal,7,200,115,10.434783,8.4


In [109]:
#getting the remaining number of wickets
innings_2_summary['player_dismissed'] = innings_2_summary['player_dismissed'].fillna(0)
innings_2_summary['player_dismissed'] = innings_2_summary['player_dismissed'].apply(lambda x:x if x == 0 else 1)
innings_2_summary['player_dismissed'] = innings_2_summary['player_dismissed'].astype('int')
wickets = innings_2_summary.groupby(by=['match_id'])['player_dismissed'].transform(lambda x: x.cumsum())
innings_2_summary['wickets left'] = 10 - wickets
innings_2_summary = innings_2_summary.reset_index(drop=True)
innings_2_summary.head()

,match_id,batting_team,bowling_team,venue,target,over,ball,total_runs,player_dismissed,dl_applied,winner,result,current_score,runs_left,balls_left,rrr,crr,wickets left
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,1,1,0,0,Sunrisers Hyderabad,normal,1,206,119,10.386555,6.0,10
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,2,0,0,0,Sunrisers Hyderabad,normal,1,206,118,10.474576,3.0,10
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,3,0,0,0,Sunrisers Hyderabad,normal,1,206,117,10.564103,2.0,10
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,4,2,0,0,Sunrisers Hyderabad,normal,3,204,116,10.551724,4.5,10
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,5,4,0,0,Sunrisers Hyderabad,normal,7,200,115,10.434783,8.4,10


In [110]:
#winner = batting team ?
def result(row):
    i=0
    if row['batting_team'] == row['winner']:
        i=2
    if row['bowling_team'] == row['winner']:
        i=1
    if row['winner']== 'To be Decided by Super-Over':
        i=0
    
    return i

In [111]:
innings_2_summary['result'] = innings_2_summary.apply(result,axis=1)
innings_2_summary['result'].astype(int)
innings_2_summary.head()

,match_id,batting_team,bowling_team,venue,target,over,ball,total_runs,player_dismissed,dl_applied,winner,result,current_score,runs_left,balls_left,rrr,crr,wickets left
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,1,1,0,0,Sunrisers Hyderabad,1,1,206,119,10.386555,6.0,10
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,2,0,0,0,Sunrisers Hyderabad,1,1,206,118,10.474576,3.0,10
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,3,0,0,0,Sunrisers Hyderabad,1,1,206,117,10.564103,2.0,10
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,4,2,0,0,Sunrisers Hyderabad,1,3,204,116,10.551724,4.5,10
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,1,5,4,0,0,Sunrisers Hyderabad,1,7,200,115,10.434783,8.4,10


In [112]:
#restricting to only positive values
innings_2_summary = innings_2_summary[innings_2_summary['balls_left'] > 0]
innings_2_summary = innings_2_summary[innings_2_summary['runs_left'] >= 0]

In [113]:
#creating the final dataset
innings_2_summary.drop(columns=['over', 'ball', 'total_runs', 'player_dismissed', 'dl_applied', 'current_score'],axis =1, inplace = True)
final = innings_2_summary

In [114]:
#dropping all null values and resetting indices
final.head()

,match_id,batting_team,bowling_team,venue,target,winner,result,runs_left,balls_left,rrr,crr,wickets left
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,119,10.386555,6.0,10
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,118,10.474576,3.0,10
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,117,10.564103,2.0,10
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,204,116,10.551724,4.5,10
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,200,115,10.434783,8.4,10


In [115]:
#saving the final dataset
final.to_csv('/home/aritrabag/Documents/internship_projects/final.csv', index = False)

In [116]:
#saving the team names
teams = final['batting_team']
teams.to_csv('/home/aritrabag/Documents/internship_projects/teams.csv', index = False)

#saving the venues
venues = final['venue']
venues.to_csv('/home/aritrabag/Documents/internship_projects/venues.csv', index = False)

#### IMPORTING THE PROCESSED DATA

In [154]:
dataset_clean_raw = pd.read_csv('/home/aritrabag/Documents/internship_projects/final.csv')

In [155]:
dataset_clean_raw.head(10)

,match_id,batting_team,bowling_team,venue,target,winner,result,runs_left,balls_left,rrr,crr,wickets left
0,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,119,10.386555,6.000000,10
1,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,118,10.474576,3.000000,10
2,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,206,117,10.564103,2.000000,10
3,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,204,116,10.551724,4.500000,10
4,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,200,115,10.434783,8.400000,10
5,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,196,114,10.315789,11.000000,10
6,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,196,113,10.407080,9.428571,10
7,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,196,112,10.500000,8.250000,10
8,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,195,111,10.540541,8.000000,10
9,1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,Sunrisers Hyderabad,1,195,110,10.636364,7.200000,10


#### SPLITTING INTO THE X(FEATURES) AND Y(TARGET)

In [156]:
targets = dataset_clean_raw['result']
dataset_clean_raw.drop(['match_id','winner','result'], axis = 1, inplace = True)
features = dataset_clean_raw.iloc[:,:]

In [157]:
features

,batting_team,bowling_team,venue,target,runs_left,balls_left,rrr,crr,wickets left
0,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,206,119,10.386555,6.000000,10
1,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,206,118,10.474576,3.000000,10
2,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,206,117,10.564103,2.000000,10
3,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,204,116,10.551724,4.500000,10
4,Royal Challengers Bangalore,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal , Hy...",207,200,115,10.434783,8.400000,10
...,...,...,...,...,...,...,...,...,...
70883,Chennai Super Kings,Mumbai Indians,"Rajiv Gandhi Intl. Cricket Stadium , Hyderabad",152,12,8,9.000000,7.500000,5
70884,Chennai Super Kings,Mumbai Indians,"Rajiv Gandhi Intl. Cricket Stadium , Hyderabad",152,10,7,8.571429,7.539823,5
70885,Chennai Super Kings,Mumbai Indians,"Rajiv Gandhi Intl. Cricket Stadium , Hyderabad",152,2,6,2.000000,7.894737,5
70886,Chennai Super Kings,Mumbai Indians,"Rajiv Gandhi Intl. Cricket Stadium , Hyderabad",152,1,5,1.200000,7.878261,5


In [158]:
targets

0        1
1        1
2        1
3        1
4        1
        ..
70883    1
70884    1
70885    1
70886    1
70887    1
Name: result, Length: 70888, dtype: int64

#### FINDING THE BEST MULTINOMIAL LOGISTIC REGRESSION PARAMETERS

In [159]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [160]:
encode = ColumnTransformer([
    ('encode',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','venue'])
]
,remainder='passthrough')

In [161]:
accuracies = []
for i in range(100):
    x_train,x_test,y_train,y_test = train_test_split(features,targets,test_size = 0.25,shuffle = True,random_state=i)
    test_predictor = Pipeline(steps=[('step1',encode), ('step2',LogisticRegression(multi_class='multinomial', solver = 'newton-cg'))])
    test_predictor.fit(x_train,y_train)
    y_predict = test_predictor.predict(x_test)
    accuracies.append(accuracy_score(y_test,y_predict))
    print('Completed trial ' + str(i+1))

Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed trial 13
Completed trial 14
Completed trial 15
Completed trial 16
Completed trial 17
Completed trial 18
Completed trial 19
Completed trial 20
Completed trial 21
Completed trial 22
Completed trial 23
Completed trial 24


/home/aritrabag/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:425: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/home/aritrabag/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/aritrabag/.local/lib/python3.10/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Completed trial 25
Completed trial 26
Completed trial 27
Completed trial 28
Completed trial 29
Completed trial 30
Completed trial 31
Completed trial 32
Completed trial 33
Completed trial 34
Completed trial 35
Completed trial 36
Completed trial 37
Completed trial 38
Completed trial 39
Completed trial 40
Completed trial 41
Completed trial 42
Completed trial 43
Completed trial 44
Completed trial 45
Completed trial 46
Completed trial 47
Completed trial 48
Completed trial 49
Completed trial 50
Completed trial 51
Completed trial 52
Completed trial 53
Completed trial 54


/home/aritrabag/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/aritrabag/.local/lib/python3.10/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Completed trial 55
Completed trial 56
Completed trial 57
Completed trial 58
Completed trial 59
Completed trial 60
Completed trial 61
Completed trial 62
Completed trial 63
Completed trial 64
Completed trial 65
Completed trial 66
Completed trial 67
Completed trial 68
Completed trial 69
Completed trial 70
Completed trial 71
Completed trial 72
Completed trial 73
Completed trial 74
Completed trial 75
Completed trial 76
Completed trial 77
Completed trial 78
Completed trial 79
Completed trial 80
Completed trial 81
Completed trial 82
Completed trial 83
Completed trial 84
Completed trial 85
Completed trial 86
Completed trial 87
Completed trial 88
Completed trial 89
Completed trial 90
Completed trial 91
Completed trial 92
Completed trial 93
Completed trial 94
Completed trial 95
Completed trial 96
Completed trial 97
Completed trial 98
Completed trial 99
Completed trial 100


In [162]:
print('The highest accuracy score is :' + str(max(accuracies)))
print('The highest r2 score is found when the random state = ' + str(accuracies.index(max(accuracies))))

The highest accuracy score is :0.7932513260354361
The highest r2 score is found when the random state = 74


#### CREATING TRAINING AND TEST SETS

In [163]:
x_train, x_test, y_train, y_test = train_test_split (features, targets, test_size = 0.25, random_state = 74 ,shuffle = True)

In [164]:
print (" The shape of x_train is : " + str(x_train.shape))
print (" The shape of y_train is : " + str(y_train.shape))

 The shape of x_train is : (53166, 9)
 The shape of y_train is : (53166,)


#### TRAINING THE PREDICTOR

In [165]:
predictor = Pipeline(steps=[
    ('step1',encode),
    ('step2',LogisticRegression(multi_class='multinomial', solver = 'newton-cg'))])

In [166]:
predictor.fit(x_train , y_train )

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team',
                                                   'venue'])])),
                ('step2',
                 LogisticRegression(multi_class='multinomial',
                                    solver='newton-cg'))])

#### TEST SET IMPLEMENTATION

In [174]:
y_predicted = predictor.predict(x_test)
y_predicted_proba = predictor.predict_proba(x_test)

#### TEST SET RESULTS

In [175]:
print('The classification test results are : \n')
print(classification_report(y_test,y_predicted))
print('Confusion Matrix :')
cm=confusion_matrix(y_test,y_predicted)
print(cm)
print('\n')
print('The test accuracy percentage is :')
accuracy =(accuracy_score(y_test,y_predicted))*100
print(accuracy)
print('\n')
print('The test precision percentage is :')
precision =(precision_score(y_test,y_predicted,average='micro'))*100
print(precision)
print('\n')
print('The test recall percentage is :')
recall =(recall_score(y_test,y_predicted,average='micro'))*100
print(recall)
print('\n')
print('The test f1 percentage is :')
f1 =(f1_score(y_test,y_predicted,average='micro'))*100
print(f1)
print('\n')
print('The test ROC AUC score is :')
roc = (roc_auc_score(y_test,y_predicted_proba,multi_class='ovr'))
print(roc)

The classification test results are : 

              precision    recall  f1-score   support

           0       0.79      0.67      0.72      2460
           1       0.79      0.79      0.79      7224
           2       0.80      0.83      0.82      8038

    accuracy                           0.79     17722
   macro avg       0.79      0.76      0.78     17722
weighted avg       0.79      0.79      0.79     17722

Confusion Matrix :
[[1643  352  465]
 [ 285 5706 1233]
 [ 161 1168 6709]]


The test accuracy percentage is :
79.32513260354361


The test precision percentage is :
79.32513260354361


The test recall percentage is :
79.32513260354361


The test f1 percentage is :
79.32513260354361


The test ROC AUC score is :
0.9225433507788471


#### SAVING THE MODEL

In [177]:
import pickle
pickle.dump(predictor,open('/home/aritrabag/Documents/internship_projects/predictor.pkl','wb'))